In [1]:
import CS_Indicators as CS
from brix import Indicator, Handler
import geopandas as gpd
import pandas as pd

In [2]:
table_name='providence_test'

### Load the Saved Data

In [3]:
geogrid=gpd.read_file('tables/{}/geogrid.geojson'.format(table_name))
zones=gpd.read_file('tables/{}/zones.geojson'.format(table_name))
zones['GEOID']=zones['GEOID'].astype(int)
zones=zones.set_index('GEOID')
simpop_df=pd.read_csv('tables/{}/simpop_df.csv'.format(table_name))

### Create and run the Cityscope model

In [5]:
H=Handler(table_name=table_name)
H.reset_geogrid_data()

In [9]:
d=CS.Density_Indicator(zones=zones)
p=CS.Proximity_Indicator(zones=zones, geogrid=geogrid)
m=CS.Mobility_indicator(zones, geogrid, table_name, simpop_df)
H.add_indicators([
    d, p, 
    m
])

# H.add_indicator(m)

Setting up Proximity Indicator
	 Getting central nodes
	 Finding all reachable geometries from each geometry
	 Calculating baseline scores
Dens Ind: 0:00:00.005466
Prox Ind: 0:00:00.105349
Prox HM: 0:00:00.005028


/Users/doorleyr/anaconda3/envs/brix/lib/python3.9/site-packages/brix/classes.py:575: UserWarning: Indicator Mobility already exists and will be overwritten
  warn(f'Indicator {indicatorName} already exists and will be overwritten')


Starting MM Update
Schedules and locations
Trip table
Route table


/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:672: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


DeckGL
Post ABM: <Response [200]>
Finished MM Update


In [ ]:
H.listen()

Starting MM Update
Schedules and locations
Trip table
Route table


/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:672: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


DeckGL
Post ABM: <Response [200]>
Finished MM Update
Dens Ind: 0:00:00.005202
Prox Ind: 0:00:00.107899
Prox HM: 0:00:00.005441
Starting MM Update
Schedules and locations
Trip table
Route table


/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:672: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


DeckGL
Post ABM: <Response [200]>
Finished MM Update
Dens Ind: 0:00:00.006239
Prox Ind: 0:00:00.109959
Prox HM: 0:00:00.004966
Starting MM Update
Schedules and locations
Trip table
Route table


/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:672: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


DeckGL
Post ABM: <Response [200]>
Finished MM Update
Dens Ind: 0:00:00.009228
Prox Ind: 0:00:00.336471
Prox HM: 0:00:00.010609
Starting MM Update
Schedules and locations
Trip table
Route table


/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:672: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


DeckGL
Post ABM: <Response [200]>
Finished MM Update
Dens Ind: 0:00:00.006785
Prox Ind: 0:00:00.261256
Prox HM: 0:00:00.009170
